# Elliptic Curves Part 1



In [4]:
def inverse_mod(a, p):
    g, x, y = extended_gcd(a, p)
    if g != 1:
        return None  # modular inverse does not exist
    else:
        return x % p

def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, x, y = extended_gcd(b % a, a)
        return g, y - (b // a) * x, x

def elliptic_curve_operations(a, b, p, P1, P2):
    def print_and_compute(expression, description, **variables):
        result = eval(expression, {'inverse_mod': inverse_mod}, variables)
        print(f"{description}: {expression} = {result}")
        return result

    x1, y1 = P1
    x2, y2 = P2

    print(f"Elliptic Curve: y^2 = x^3 + {a}x + {b} (mod {p})")
    print(f"P1 = {P1}, P2 = {P2}")

    if P1 == P2:
        print("Case: P1 = P2")
        g = gcd(2 * y1, p)
        if g == 1:
            m = print_and_compute('(3 * x1 ** 2 + a) * inverse_mod(2 * y1, p)', 'Slope m', x1=x1, y1=y1, a=a, p=p)
        else:
            print(f"Modular inverse does not exist for 2*y1; gcd is {g}. Operation cannot proceed.")
            return
    else:
        print("Case: P1 != P2")
        delta_x = x2 - x1
        g = gcd(delta_x, p)
        if g == 1:
            m = print_and_compute('(y2 - y1) * inverse_mod(delta_x, p)', 'Slope m', x1=x1, y1=y1, x2=x2, y2=y2, p=p, delta_x=delta_x)
        else:
            print(f"Modular inverse does not exist for x2 - x1; gcd({delta_x}, {p}) is {g}. Operation cannot proceed.")
            return

    x3 = print_and_compute('(m**2 - x1 - x2) % p', 'x3', m=m, x1=x1, x2=x2, p=p)
    y3 = print_and_compute('(m * (x1 - x3) - y1) % p', 'y3', m=m, x1=x1, x3=x3, y1=y1, p=p)
    print(f"The sum P1 + P2 = ({x3 % p}, {y3 % p}) (mod {p})")

# Example usage:
# For the curve y^2 = x^3 - 2 (mod 7), and points P1 = (3, 2) and P2 = (5, 5)
#elliptic_curve_operations(0, -2, 7, (3, 2), (5, 5))

# For the curve y^2 = x^3 + 3 (mod 7), and points P1 = (1, 2) and P2 = (6, 3)
#elliptic_curve_operations(0, 3, 7, (1, 2), (6, 3))

# Test case with mod 35
elliptic_curve_operations(0, 5, 19, (1, 14), (1, 14))


Elliptic Curve: y^2 = x^3 + 0x + 5 (mod 19)
P1 = (1, 14), P2 = (1, 14)
Case: P1 = P2
Slope m: (3 * x1 ** 2 + a) * inverse_mod(2 * y1, p) = 51
x3: (m**2 - x1 - x2) % p = 15
y3: (m * (x1 - x3) - y1) % p = 13
The sum P1 + P2 = (15, 13) (mod 19)


## Working with Elliptic Curves



12.2.3


In [5]:
# Sagemath program to find an elliptic curve for a point (x, y) mod p

# Input point and mod value
x = 43
y = 21
p = 101

# Initial equation: y^2 = x^3 + bx + c (mod p)
# y^2 - x^3 = bx + c (mod p)
lhs = mod(y^2 - x^3, p)

print(f"Left-hand side (lhs) after computing (y^2 - x^3) mod {p}: {lhs}")

# Initialize flag for multiple combinations
multiple_combinations = False

# Case 1: Choose b = 0, solve for c
b1 = 0
c1 = lhs
print(f"When b=0, c={c1} and the curve is y^2 = x^3 + {c1} mod {p}")

# Case 2: Choose c = 0, solve for b
c2 = 0

# Check if gcd(x, p) = 1, i.e., x has a modular inverse mod p
if gcd(x, p) == 1:
    multiple_combinations = True
    x_inv = inverse_mod(x, p)
    b2 = mod(lhs * x_inv, p)
    print(f"When c=0, b={b2} and the curve is y^2 = x^3 + {b2}x mod {p}")
else:
    print("The modular inverse does not exist; gcd(x, p) != 1.")

if not multiple_combinations:
    print("Multiple combinations of b and c are not possible.")
else:
    print("Multiple combinations of b and c are possible.")


Left-hand side (lhs) after computing (y^2 - x^3) mod 101: 17
When b=0, c=17 and the curve is y^2 = x^3 + 17 mod 101
When c=0, b=92 and the curve is y^2 = x^3 + 92x mod 101
Multiple combinations of b and c are possible.


## Activity 3: Finding Points on Elliptic Curves



In [7]:
# Import the relevant modules
from sage.rings.finite_rings.integer_mod_ring import IntegerModRing

# Set up the modulus and equation
modulus = 13
x_value = 3
rhs_value = x_value^3 + (7*x_value) + 8  # This is x^3 + 3 + 12

# Create the mod ring
R = IntegerModRing(modulus)

# Calculate the value under the square root
value_to_sqrt = R(rhs_value)

# Try to find the square root(s)
try:
    sqrt_values = value_to_sqrt.sqrt(all=True)
    points = [(x_value, y) for y in sqrt_values]
except ValueError:
    points = []

# Print out the points (if any)
print("The points (1, y) on the curve are:", points)


The points (1, y) on the curve are: [(3, 2), (3, 11)]


In [8]:
from sage.rings.finite_rings.integer_mod_ring import IntegerModRing

# Initialize variables
modulus = 13  # the field F_p
a = 0  # curve parameter a
b = 6  # curve parameter b

# Create the mod ring
R = IntegerModRing(modulus)

# Initialize list to hold points
all_points = []

# Loop through all possible x values
for x_value in range(modulus):
    # Compute the right-hand side of the equation
    rhs_value = x_value^3 + (a * x_value) + b

    # Calculate the value under the square root
    value_to_sqrt = R(rhs_value)

    # Try to find the square root(s)
    try:
        sqrt_values = value_to_sqrt.sqrt(all=True)
        points = [[x_value, y] for y in sqrt_values]
        all_points.extend(points)
    except ValueError:
        continue

# Output the points
print("The points on the curve are:", all_points)


The points on the curve are: [[2, 1], [2, 12], [5, 1], [5, 12], [6, 1], [6, 12]]


### 12.3.2 part a



In [9]:
def generate_prime(bits=16):
    """
    Generate a random prime number of the specified number of bits using SageMath.
    """
    while True:
        p = random_prime(2**(bits-1), 2**bits - 1)
        return p

def generate_elliptic_curve(x, y):
    """
    Generate an elliptic curve of the form y^2 = x^3 + ax + b (mod p) given a point (x, y).
    """
    p = generate_prime(bits=16)  # Generate a random prime number with 128 bits

    # Choose random values for a and b
    a = randint(1, p // 2)
    b = randint(1, p // 2)

    # Ensure that (x, y) satisfies the elliptic curve equation
    if y**2 % p != (x**3 + a*x + b) % p:
        a = randint(1, p // 2)
        b = randint(1, p // 2)

    return a, b, p

# Example usage:
x = 5  # Replace with your desired x-coordinate
y = 7  # Replace with your desired y-coordinate

a, b, p = generate_elliptic_curve(x, y)
print(f"Generated elliptic curve: y^2 = x^3 + {a}x + {b} (mod {p})")

Generated elliptic curve: y^2 = x^3 + 237x + 1377 (mod 4201)


### part b



In [10]:
def generate_prime(bits=16):
    """
    Generate a random prime number of the specified number of bits.
    """
    while True:
        p = randint(2**(bits-1), 2**bits - 1)
        if all(p % i != 0 for i in range(2, int(p ** 0.5) + 1)):
            return p

def add_points(a, b, p, P):
    x1, y1 = P
    if y1 == 0:
        return "Point at Infinity"
    
    m = (3 * x1**2 + a) * pow(2 * y1, -1, p)
    
    xR = (m**2 - 2 * x1) % p
    yR = (m * (x1 - xR) - y1) % p
    
    return (xR, yR)

def generate_elliptic_curve(x, y):
    """
    Generate an elliptic curve and a second point Q.
    """
    p = generate_prime(bits=16)

    a = randint(1, p // 2)
    b = randint(1, p // 2)

    while y**2 % p != (x**3 + a*x + b) % p:
        a = randint(1, p // 2)
        b = randint(1, p // 2)
        
    Q = add_points(a, b, p, (x, y))

    return a, b, p, Q

x = 5
y = 7

a, b, p, Q = generate_elliptic_curve(x, y)
print(f"Generated elliptic curve: y^2 = x^3 + {a}x + {b} (mod {p})")
print(f"Second point Q: {Q}")


Generated elliptic curve: y^2 = x^3 + 17811x + 24651 (mod 56891)
Second point Q: (46197, 23017)


### Extended \- both together with an option for a manual Q.



In [11]:
def generate_prime(bits=16):
    """
    Generate a random prime number of the specified number of bits.
    """
    while True:
        p = randint(2**(bits-1), 2**bits - 1)
        if all(p % i != 0 for i in range(2, int(p ** 0.5) + 1)):
            return p

def generate_elliptic_curve(x, y):
    """
    Generate an elliptic curve and a second point Q.
    """
    p = generate_prime(bits=16)
    a = randint(1, p // 2)
    b = randint(1, p // 2)

    while y**2 % p != (x**3 + a*x + b) % p:
        a = randint(1, p // 2)
        b = randint(1, p // 2)
    
    Q = add_points_on_curve(p, a, b, (x, y), (x, y))
    
    return a, b, p, Q

def add_points_on_curve(p, a, b, P, Q):
    """
    Add two points P and Q on an elliptic curve y^2 = x^3 + ax + b (mod p).
    """
    x1, y1 = P
    x2, y2 = Q
    
    if (pow(y1, 2, p) - (pow(x1, 3, p) + a * x1 + b)) % p != 0 or \
       (pow(y2, 2, p) - (pow(x2, 3, p) + a * x2 + b)) % p != 0:
        return "Error: One or both points are not on the curve."
    
    if x1 == x2 and y1 == -y2:
        return "Point at infinity"
    
    if P == Q and y1 == 0:
        return "Point at infinity"
    
    if P != Q:
        m = (y2 - y1) * pow(x2 - x1, -1, p)
    else:
        m = (3 * pow(x1, 2, p) + a) * pow(2 * y1, -1, p)
        
    xR = (pow(m, 2, p) - x1 - x2) % p
    yR = (-y1 + m * (x1 - xR)) % p
    
    return (xR, yR)

# Example usage:
x = 5  # Replace with your point P coordinates
y = 7  # Replace with your point Q coordinates

a, b, p, Q = generate_elliptic_curve(x, y)
print(f"Generated elliptic curve: y^2 = x^3 + {a}x + {b} (mod {p})")
print(f"Second point Q: {Q}")

result = add_points_on_curve(p, a, b, (x, y), Q)
print(f"Result of addition: {result}")


Generated elliptic curve: y^2 = x^3 + 4999x + 8726 (mod 33797)
Second point Q: (28574, 16622)
Result of addition: (6968, 17823)


12.3.2


In [12]:
def generate_prime(bits=16):
    """
    Generate a random prime number of the specified number of bits.
    """
    while True:
        p = randint(2**(bits-1), 2**bits - 1)
        if all(p % i != 0 for i in range(2, int(p ** 0.5) + 1)):
            return p

def generate_elliptic_curve(x, y, manual_Q=None):
    """
    Generate an elliptic curve and a second point Q. If manual_Q is provided, use it as Q.
    """
    p = generate_prime(bits=16)
    a = randint(1, p // 2)
    b = randint(1, p // 2)

    while y**2 % p != (x**3 + a*x + b) % p:
        a = randint(1, p // 2)
        b = randint(1, p // 2)
    
    if manual_Q:
        Q = manual_Q
    else:
        Q = add_points_on_curve(p, a, b, (x, y), (x, y))
    
    return a, b, p, Q

def add_points_on_curve(p, a, b, P, Q):
    """
    Add two points P and Q on an elliptic curve y^2 = x^3 + ax + b (mod p).
    """
    x1, y1 = P
    x2, y2 = Q
    
    if (pow(y1, 2, p) - (pow(x1, 3, p) + a * x1 + b)) % p != 0 or \
       (pow(y2, 2, p) - (pow(x2, 3, p) + a * x2 + b)) % p != 0:
        return "Error: One or both points are not on the curve."
    
    if x1 == x2 and y1 == -y2:
        return "Point at infinity"
    
    if P == Q and y1 == 0:
        return "Point at infinity"
    
    if P != Q:
        m = (y2 - y1) * pow(x2 - x1, -1, p)
    else:
        m = (3 * pow(x1, 2, p) + a) * pow(2 * y1, -1, p)
        
    xR = (pow(m, 2, p) - x1 - x2) % p
    yR = (-y1 + m * (x1 - xR)) % p
    
    return (xR, yR)

# Example usage with generated Q:
x = 2
y = 19
a, b, p, Q = generate_elliptic_curve(x, y)
print(f"Generated elliptic curve: y^2 = x^3 + {a}x + {b} (mod {p})")
print(f"Generated point Q: {Q}")
result = add_points_on_curve(p, a, b, (x, y), Q)
print(f"Result of addition: {result}")

# Example usage with manual Q:
manual_Q = (24181, 22373)
a, b, p, Q = generate_elliptic_curve(x, y, manual_Q)
print(f"Generated elliptic curve: y^2 = x^3 + {a}x + {b} (mod {p})")
print(f"Manual point Q: {Q}")
result = add_points_on_curve(p, a, b, (x, y), Q)
print(f"Result of addition: {result}")

Generated elliptic curve: y^2 = x^3 + 76x + 201 (mod 41213)
Generated point Q: (30483, 3143)
Result of addition: (15446, 39745)


Generated elliptic curve: y^2 = x^3 + 24456x + 4378 (mod 52937)
Manual point Q: (24181, 22373)
Result of addition: Error: One or both points are not on the curve.
